In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/My\ Drive/TRVQA

# List files to make sure we're in the expected directory.
# Your output will look different, showing your own Drive files here.
!ls

/content/drive/My Drive/TRVQA
benchmark		       circuit.py     __init__.py   utils
bench_tsp_shot_diff_25_real    ex_value_pots  measure
bench_tsp_shot_diff_4816_real  gates	      optimization
bench_tsp_shot_diff_49_real    hamiltonian    __pycache__


In [ ]:
import sys
sys.path.append('/content/drive/My\ Drive/TRVQA')


<>:2: SyntaxWarning: invalid escape sequence '\ '
<>:2: SyntaxWarning: invalid escape sequence '\ '
/tmp/ipython-input-691171030.py:2: SyntaxWarning: invalid escape sequence '\ '
  sys.path.append('/content/drive/My\ Drive/TRVQA')


In [ ]:
!pip install qiskit
!pip install gurobipy
!pip install qiskit_optimization

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.0/8.0 MB 114.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 115.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.3/14.3 MB 136.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 646.5/646.5 kB 48.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 237.1/237.1 kB 20.9 MB/s eta 0:00:00
  Created wheel for docplex: filename=docplex-2.30.251-py3-none-any.whl size=685954 sha256=f1132f5f39b9d58c0b6d7f62549f451e573512f6ca936807c01c8d19c21c0c4e
  Stored in directory: /root/.cache/pip/wheels/c1/d9/5c/1d919b1e441ebbaff244513a13d09c3c0378401601d4184bb0
Successfully built docplex


In [ ]:
import sys
import os
import sys
import os
# Add the parent directory of TRVQA to sys.path
from measure.enums import MeasureMethod
from optimization.gradients.vanilla_parameter_shift import vanilla_parameter_shift
from optimization.optimization import minimize, minimize_custom
#from utils.maxcut import gengraph, create_hamiltonian, make_hamiltonian
import torch
from torch.autograd import Function
from qiskit.circuit.library import QAOAAnsatz
from qiskit.quantum_info import SparsePauliOp, Statevector
from qiskit import QuantumCircuit

import numpy as np
import torch
from circuit import Circuit

def compute_maxcut_value(bitstring, graph):
    """Computes the MaxCut value for a given bitstring solution."""
    cut_value = sum(w for (u,v), w in graph if bitstring[u] != bitstring[v])
    return cut_value

# for s in best_value:
#     print(compute_maxcut_value(s, g))

from pyparsing import Optional
import torch
import numpy as np
import matplotlib.pyplot as plt
from qiskit import QuantumCircuit
from qiskit.quantum_info import Statevector, Operator, SparsePauliOp
from optimization.optimization import minimize, minimize_custom
# TRVQA imports

from circuit import Circuit
from hamiltonian.hamiltonian import Hamiltonian
from measure.enums import MeasureMethod
torch.cuda.empty_cache()
import numpy as np
import torch
from qiskit import QuantumCircuit
from qiskit.quantum_info import Statevector, Operator, SparsePauliOp
import torch

from measure.enums import MeasureMethod
from optimization.optimization import minimize
from utils.maxcut import gengraph, create_hamiltonian,make_hamiltonian
from hamiltonian.hamiltonian import Hamiltonian
import numpy as np
import torch
from circuit import Circuit
import time


In [ ]:
import gurobipy as gp
from gurobipy import GRB
import networkx as nx

def maxcut_value(G) -> float:
    """
    Solve weighted Max-Cut on an undirected NetworkX graph using Gurobi.

    Args:
        G: networkx.Graph
           Undirected graph. Edge weights read from attribute 'weight' (default 1.0).

    Returns:
        float: Optimal Max-Cut value.
    """
    # if not isinstance(G, nx.Graph) or G.is_directed():
    #     raise ValueError("G must be an undirected networkx.Graph")

    model = gp.Model("MaxCut")
    model.Params.OutputFlag = 0  # silence solver output

    # Binary variable x[i] ∈ {0,1} indicates partition side of node i
    #nodes = list(G.nodes())
    # Number of nodes
    nodes = set()
    for e,w in G:
        u,v = e
        nodes.add(u)
        nodes.add(v)
    n = len(nodes)

    x = model.addVars(nodes, vtype=GRB.BINARY, name="x")
    # Objective: Maximize the cut value
    model.setObjective(
        gp.quicksum(w * (x[u] + x[v] - 2 * x[u] * x[v]) for (u, v), w in G),
        GRB.MAXIMIZE
    )

    # Optimize the model
    model.optimize()


    if model.Status != GRB.OPTIMAL:
        raise RuntimeError(f"Gurobi did not find an optimal solution (status={model.Status}).")

    return float(model.ObjVal)
import json

N_list = [4, 8, 16, 24, 36, 52, 70, 90, 100]
from dataclasses import dataclass, asdict
from typing import Dict, List, Tuple, Callable, Any
import torch, numpy as np

# --- Result container ---------------------------------------------------------
@dataclass
class RunLog:
    exp_values: List[float]
    best_results: List[Any]   # bitstrings, ints, etc.
    iter_times: List[float]
    final_theta: torch.Tensor

# --- Helpers ------------------------------------------------------------------
# --- Helpers ------------------------------------------------------------------
def build_qiskit_like_hamiltonian(hm):
    """hm is your [(coeff, bool_list), ...]"""
    pauli_strings, coefficients = [], []
    for elm in hm:
        coeff, pauli = elm.coeffs[0], elm.paulis[0]

        pauli_strings.append(str(pauli[::-1]))
        coefficients.append(float(coeff))
    # print(pauli_strings)
    # print(coefficients)
    return Hamiltonian(len(pauli_strings[0]), pauli_strings, coefficients)
def build_circuit(N: int, D: int, rank: int = 10, device: str = 'cuda', isRing: bool = True) -> Circuit:
    c = Circuit(N, device=device)
    for _ in range(D):
        for j in range(N): c.rx(j)
        for j in range(N): c.rz(j)
        for j in range(N-1): c.cx(j, j+1)
        if isRing:
            c.cx(N-1, 0)
    c.rank = rank
    return c

# --- Method runners -----------------------------------------------------------
def run_statevector_vqe(N: int, D: int, h, iter_steps: int, lr: float = 1e-2, theta0=None, isRing=True) -> RunLog:
    model = QuantumCircuitTorch(theta0, N, D, h, isRing=isRing)
    opt = torch.optim.SGD(model.parameters(), lr=lr)

    exp_vals, best_results, iter_times = [], [], []
    for _ in range(iter_steps):
        opt.zero_grad()
        out = model()                   # expectation value (loss)
        out.backward()
        opt.step()
        exp_vals.append(float(out.item()))
        best_results.append(get_opt_cut_sv(model.theta, N, D, h, isRing))
        # if you track iteration time, append here (e.g., time.perf_counter deltas)
    return RunLog(exp_values=exp_vals, best_results=best_results, iter_times=iter_times, final_theta=model.theta.detach().clone())

def run_tr_vqe(method, circuit, theta0, hamil, iter_steps: int, opt, grad_kwargs: dict) -> RunLog:
    """Generic TR-VQE using your Optimizer + ParameterShift flavors."""
    # # Build gradient object with provided kwargs, override measure method:
    from optimization.gradients.batch_parameter_shift import BatchParameterShiftGradient
    from optimization.optimizer import Optimizer
    grad = BatchParameterShiftGradient(
        torch.pi/2,
        grad_kwargs.get("batches", 100),
        grad_kwargs.get("shots", int(1e3)),
        method,                      # MeasureMethod
        grad_kwargs.get("D", 1)
    )
    #opt = Optimizer(torch.optim.AdamW, {'lr': lr})
    #opt = Optimizer(torch.optim.AdamW, {'lr': lr})
    # opt = Optimizer(torch.optim.RMSprop, {
    #     "lr": 0.05,      # 0.03–0.04 works best at N=8
    #     "alpha": 0.95,     # smoother EWMA than 0.9
    #     "eps": 1e-8,
    #     # "momentum": 0.9,   # NEW
    #     # "centered": True,  # NEW
    #     "weight_decay": 0.0,
    # })
    theta_init = theta0.clone().detach().requires_grad_(True)

    new_theta, exp_vals, best_results, iter_times = minimize(
        circuit, theta_init, hamil, opt, grad, iter_steps, best_value_method="argmax_tr_noinv_LE"
    )
    return RunLog(
        exp_values=[float(v.real) for v in exp_vals],
        best_results=list(best_results),
        iter_times=list(iter_times),
        final_theta=new_theta.detach().clone()
    )


In [ ]:
# =========================
# Resume-in-place utilities (with optimizer_name + optimizer object creation)
# =========================
import os, csv, json
import torch
import torch.optim as optim

# ---------- Summary utilities ----------

def _read_summary(summary_csv="bench_tsp_shot_diff_49_real/summary.csv"):
    with open(summary_csv, "r") as f:
        return list(csv.DictReader(f))

def _write_summary_header_if_missing(summary_csv="bench_tsp_shot_diff_49_real/summary.csv"):
    if not os.path.exists(summary_csv):
        with open(summary_csv, "w", newline="") as f:
            csv.writer(f).writerow([
                "run_id","method","N","depth","seed",
                "theta_path","iter_csv","optimizer_name", "shots"   # ✅ optimizer column
            ])

def _find_run_row(
    method: str,
    N: int,
    D: int,
    seed: int,
    optimizer_name: str | None = None,
    shots = None,
    summary_csv="bench_tsp_shot_diff_49_real/summary.csv"
):
    """Return the LAST matching summary row (dict) or None."""
    if not os.path.exists(summary_csv):
        return None
    rows = _read_summary(summary_csv)
    cand = [
        r for r in rows
        if r["method"] == method
        and int(r["N"]) == N
        and int(r["depth"]) == D
        and int(r["seed"]) == seed
        and (optimizer_name is None or r.get("optimizer_name", "") == optimizer_name)
        and int(r['shots']) == shots
    ]
    return cand[-1] if cand else None

def _read_last_iter(iter_csv_path: str) -> int:
    """Return last iteration index in iter CSV, or -1 if file empty."""
    if not os.path.exists(iter_csv_path):
        return -1
    last = -1
    with open(iter_csv_path, "r") as f:
        r = csv.DictReader(f)
        for row in r:
            last = int(row["iter"])
    return last

# ---------- Append run ----------

def append_run_inplace(
    runlog: "RunLog",
    *,
    method: str,
    N: int,
    D: int,
    seed: int,
    optimizer_name: str = "",
    outdir: str = "bench_tsp_shot_diff_49_real",
    summary_csv: str = "bench_tsp_shot_diff_49_real/summary.csv",
    shots = None
):
    """
    Append new iterations to the *existing* run for (method, N, D, seed, optimizer_name),
    and overwrite its final theta file. If not found, creates a new one.
    """
    os.makedirs(outdir, exist_ok=True)
    _write_summary_header_if_missing(summary_csv)

    row = _find_run_row(method, N, D, seed, optimizer_name,shots, summary_csv)

    if row is None:
        import uuid
        run_id = str(uuid.uuid4())[:8]
        theta_path = os.path.join(outdir, f"theta_{run_id}.pt")
        iter_csv_path = os.path.join(outdir, f"iter_{run_id}.csv")
        with open(iter_csv_path, "w", newline="") as f:
            w = csv.writer(f); w.writerow(["iter","exp_value","best_result","iter_time"])
        with open(summary_csv, "a", newline="") as f:
            w = csv.writer(f)
            w.writerow([run_id, method, N, D, seed, theta_path, iter_csv_path, optimizer_name, shots])
    else:
        run_id        = row["run_id"]
        theta_path    = row["theta_path"]
        iter_csv_path = row["iter_csv"]
        if optimizer_name and row.get("optimizer_name", "") != optimizer_name:
            rows = _read_summary(summary_csv)
            for r in rows:
                if r["run_id"] == run_id:
                    r["optimizer_name"] = optimizer_name
            with open(summary_csv, "w", newline="") as f:
                writer = csv.DictWriter(f, fieldnames=rows[0].keys())
                writer.writeheader()
                writer.writerows(rows)

    start_iter = _read_last_iter(iter_csv_path) + 1

    with open(iter_csv_path, "a", newline="") as f:
        w = csv.writer(f)
        for i, (e, b, t) in enumerate(zip(runlog.exp_values, runlog.best_results, runlog.iter_times)):
            w.writerow([start_iter + i, e, json.dumps(b), t])

    torch.save(runlog.final_theta.detach().cpu(), theta_path)

    return {
        "run_id": run_id,
        "theta_path": theta_path,
        "iter_csv": iter_csv_path,
        "start_iter": start_iter
    }

# ---------- Helper: create optimizer ----------

def _make_optimizer(name: str, params, lr: float):
    """
    Construct optimizer object from name string.
    Extend here for more optimizers easily.
    """
    name = name.lower()
    if name == "adam":
        return optim.Adam([params], lr=lr)
    elif name == "adamw":
        return optim.AdamW([params], lr=lr)
    elif name == "sgd":
        return optim.SGD([params], lr=lr, momentum=0.9)
    elif name == "rmsprop":
        return optim.RMSprop([params], lr=lr)
    else:
        raise ValueError(f"Unknown optimizer: {name}")

# ---------- Resume runner ----------

def benchmark_resume_inplace(
    method: str,
    edges,
    N: int,
    D: int,
    extra_iter_steps: int,
    *,
    seed: int,
    optimizer_name: str,
    optimizer,
    theta_fallback: torch.Tensor,   # used only if no prior run exists
    lr: float = 1e-2,               # ✅ unified learning rate
    rank: int = 10,
    isRing: bool = True,
    shots: int | None = None,
    batch_size_ef: int = 200,
    batch_size_cs: int = 300,
    batch_size_ps: int = 200,
    outdir: str = "bench_tsp_shot_diff_49_real"
):
    """
    Continue training the SAME run (method, N, D, seed, optimizer_name).
    Creates optimizer internally and passes it to the runner.
    """
    # set seeds
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

    # try loading last theta (if run exists)
    row = _find_run_row(method, N, D, seed, optimizer_name, shots, os.path.join(outdir, "summary.csv"))
    if row is not None and os.path.exists(row["theta_path"]):
        theta0 = torch.load(row["theta_path"], map_location="cpu").detach()
        print(f"[resume] Loaded previous theta for ({method}, N={N}, seed={seed}, optimizer={optimizer_name}).")
    else:
        theta0 = theta_fallback.clone().detach()
        print(f"[resume] No previous run found; starting fresh for ({method}, N={N}, seed={seed}, optimizer={optimizer_name}).")
    temp_tsp = Tsp(edges)
    qp = temp_tsp.to_quadratic_program()
    qp2qubo = QuadraticProgramToQubo()
    qubo = qp2qubo.convert(qp)
    qubitOp, offset = qubo.to_ising()
    # build H and circuit (your helpers)
    h_qsv = qubitOp
    h_tr  = build_qiskit_like_hamiltonian(qubitOp)
    circuit = build_circuit(N, D, rank=rank, device="cuda", isRing=isRing)

    # ✅ Create optimizer here
    theta0 = theta0.clone().detach().requires_grad_(True)
    #optimizer = _make_optimizer(optimizer_name, theta0, lr)

    # select measurement method
    mm = {
        "tr_efficient": MeasureMethod.EFFICIENT_CONTRACTION,
        "tr_correct_sampling": MeasureMethod.CORRECT_SAMPLING,
        "tr_sampling": MeasureMethod.SAMPLING,
    }[method]
    if mm == MeasureMethod.CORRECT_SAMPLING:
        grad_common = dict(batches=batch_size_cs, shots=(shots or 10_000), D=D, isRing=isRing)
    elif mm == MeasureMethod.SAMPLING:
        grad_common = dict(batches=batch_size_ps, shots=(shots or 10_000), D=D, isRing=isRing)
    else:
        grad_common = dict(batches=batch_size_ef, shots=(shots or 10_000), D=D, isRing=isRing)

    # ✅ Pass optimizer into your training loop
    runner = lambda: run_tr_vqe(
        mm,
        circuit,
        theta0,
        h_tr,
        extra_iter_steps,
        optimizer,
        grad_common
    )

    # run extra steps
    log: RunLog = runner()

    # append-in-place + overwrite theta
    info = append_run_inplace(
        log,
        method=method,
        N=N,
        D=D,
        seed=seed,
        optimizer_name=optimizer_name,
        outdir=outdir,
        shots=shots
    )
    print(f"[resume] Appended {len(log.exp_values)} iters starting from {info['start_iter']}.")
    return info, log


In [ ]:
# Load initial parameters
theta_init_by_seed = []
for i in range(10):
  a = torch.load(f"/content/drive/MyDrive/TRVQA/benchmark/theta_inits/theta_init_s{i}.pt")
  b = torch.load(f"/content/drive/MyDrive/TRVQA/benchmark/theta_inits/theta_init_s{i}_extra.pt")
  theta_init_by_seed.append(torch.cat([a,b]))

In [ ]:
with open("benchmark/experiment_grid_tsp.json", "r") as f:
    loaded = json.load(f)

def keys_to_int(d):
    """Recursively convert all string keys that look like integers to int."""
    if isinstance(d, dict):
        new_dict = {}
        for k, v in d.items():
            # Convert numeric string keys to int
            new_key = int(k) if isinstance(k, str) and k.isdigit() else k
            new_dict[new_key] = keys_to_int(v)
        return new_dict
    else:
        return d

exp_grid_loaded = [keys_to_int(entry) for entry in loaded]

print(exp_grid_loaded[50]['G'])  # should match your original structure


{0: {1: {'weight': 38.0}, 2: {'weight': 13.0}, 3: {'weight': 24.0}, 4: {'weight': 73.0}, 5: {'weight': 47.0}, 6: {'weight': 32.0}}, 1: {0: {'weight': 38.0}, 2: {'weight': 50.0}, 3: {'weight': 28.0}, 4: {'weight': 42.0}, 5: {'weight': 57.0}, 6: {'weight': 6.0}}, 2: {0: {'weight': 13.0}, 1: {'weight': 50.0}, 3: {'weight': 32.0}, 4: {'weight': 82.0}, 5: {'weight': 53.0}, 6: {'weight': 44.0}}, 3: {0: {'weight': 24.0}, 1: {'weight': 28.0}, 2: {'weight': 32.0}, 4: {'weight': 51.0}, 5: {'weight': 65.0}, 6: {'weight': 22.0}}, 4: {0: {'weight': 73.0}, 1: {'weight': 42.0}, 2: {'weight': 82.0}, 3: {'weight': 51.0}, 5: {'weight': 99.0}, 6: {'weight': 45.0}}, 5: {0: {'weight': 47.0}, 1: {'weight': 57.0}, 2: {'weight': 53.0}, 3: {'weight': 65.0}, 4: {'weight': 99.0}, 6: {'weight': 55.0}}, 6: {0: {'weight': 32.0}, 1: {'weight': 6.0}, 2: {'weight': 44.0}, 3: {'weight': 22.0}, 4: {'weight': 45.0}, 5: {'weight': 55.0}}}


In [ ]:
N_list_tsp = [4, 9, 16, 25, 36, 49, 64, 81, 100]  # your Ns
print(N_list_tsp[5])
from qiskit_optimization.applications import Tsp
import networkx as nx
from qiskit_optimization.converters import QuadraticProgramToQubo

49


In [ ]:
from optimization.optimizer import Optimizer
import torch

optimizer_configs = [
    {
        "name": "RMSprop_003",
        "wrapper": Optimizer(torch.optim.RMSprop, {
            "lr": 0.05,
            "alpha": 0.95,
            "eps": 1e-8,
            "weight_decay": 0.0,
        })
    },
]


In [ ]:
for target_N in [49]:
  graph_lookup = {(g['N'], g['graph_id']): g['G'] for g in exp_grid_loaded}
  for D in [6,5,4,3,2,1]:
      for seed in [0,1,2]:
        for shots in [int(1e3)]:
          g = graph_lookup[(target_N, seed)]
          N = target_N
          param_size = 2 * D * N
          theta0 = theta_init_by_seed[seed][:param_size]
          #for meth in ["tr_correct_sampling"]:
          for meth in ["tr_efficient" ,"tr_correct_sampling"]:
              for opt_cfg in optimizer_configs:
                  # Create parameter tensor
                  theta = theta0.clone().detach().requires_grad_(True)

                  # ✅ Build optimizer object from wrapper
                  optimizer = opt_cfg["wrapper"]

                  print(f"[Graph {seed}] D={D}, seed={seed}, N={N}, "
                        f"method={meth}, optimizer={opt_cfg['name']}")

                  info, log = benchmark_resume_inplace(
                      method=meth,
                      edges=g,
                      N=N,
                      D=D,
                      extra_iter_steps=100,
                      seed=seed,
                      optimizer_name=opt_cfg["name"],  # logged in summary.csv
                      optimizer=optimizer,             # actual optimizer object
                      theta_fallback=theta,
                      rank=10,
                      isRing=True,
                      #lr_tr=0.1,  # for convenience
                      shots=shots,
                      outdir="bench_tsp_shot_diff_49_real",
                      batch_size_ef=200,
                      batch_size_cs=700,
                      batch_size_ps=1000,
                  )

                  torch.cuda.empty_cache()
                  print("updated files:", info)


[Graph 0] D=6, seed=0, N=49, method=tr_efficient, optimizer=RMSprop_003
[resume] No previous run found; starting fresh for (tr_efficient, N=49, seed=0, optimizer=RMSprop_003).


/tmp/ipython-input-764368160.py:71: ComplexWarning: Casting complex values to real discards the imaginary part
  coefficients.append(float(coeff))


Progress: [=============================>] 99% | ETA: 00:32 | Loss: -149807.6406[resume] Appended 100 iters starting from 0.
updated files: {'run_id': '2cdea0d8', 'theta_path': 'bench_tsp_shot_diff_49_real/theta_2cdea0d8.pt', 'iter_csv': 'bench_tsp_shot_diff_49_real/iter_2cdea0d8.csv', 'start_iter': 0}
[Graph 0] D=6, seed=0, N=49, method=tr_correct_sampling, optimizer=RMSprop_003
[resume] Loaded previous theta for (tr_correct_sampling, N=49, seed=0, optimizer=RMSprop_003).


/tmp/ipython-input-764368160.py:71: ComplexWarning: Casting complex values to real discards the imaginary part
  coefficients.append(float(coeff))


Progress: [=============================>] 99% | ETA: 00:18 | Loss: -7068.8074[resume] Appended 100 iters starting from 100.
updated files: {'run_id': '65f6a5e6', 'theta_path': 'bench_tsp_shot_diff_49_real/theta_65f6a5e6.pt', 'iter_csv': 'bench_tsp_shot_diff_49_real/iter_65f6a5e6.csv', 'start_iter': 100}
[Graph 1] D=6, seed=1, N=49, method=tr_efficient, optimizer=RMSprop_003
[resume] No previous run found; starting fresh for (tr_efficient, N=49, seed=1, optimizer=RMSprop_003).
Progress: [=============================>] 99% | ETA: 00:32 | Loss: -144806.0000[resume] Appended 100 iters starting from 0.
updated files: {'run_id': 'a86cce2d', 'theta_path': 'bench_tsp_shot_diff_49_real/theta_a86cce2d.pt', 'iter_csv': 'bench_tsp_shot_diff_49_real/iter_a86cce2d.csv', 'start_iter': 0}
[Graph 1] D=6, seed=1, N=49, method=tr_correct_sampling, optimizer=RMSprop_003
[resume] No previous run found; starting fresh for (tr_correct_sampling, N=49, seed=1, optimizer=RMSprop_003).
Progress: [=============

In [ ]:
84

84

In [ ]:
torch.cuda.empty_cache()
